In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from keras.models import load_model
import math

In [38]:
df_testing= pd.read_csv('..\SacredTemplate\data\database.csv')
df_training=pd.read_excel('..\SacredTemplate\data\database_new.xlsx')

X_train = df_training.iloc[:, :8]
Y_train = df_training.iloc[:, 25:28]
X_test = df_testing.loc[:, ['wavelength', 'fractal_dimension', 'fraction_of_coating', 'primary_particle_size',
                   'number_of_primary_particles',
                   'vol_equi_radius_outer', 'vol_equi_radius_inner', 'equi_mobility_dia']]
# Normalizaing Min max
scaling_x = MinMaxScaler()
scaling_y = MinMaxScaler()
X_train = scaling_x.fit_transform(X_train)
X_test = scaling_x.transform(X_test)
Y_train = scaling_y.fit_transform(Y_train)

model = load_model('best_model.hdf5')
Y_test = model.predict(X_test)
Y_test = scaling_y.transform(Y_test)

C:\Users\JK\anaconda3\envs\TUK\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [42]:
np.zeros_like(df_testing['wavelength'])+2

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [59]:
#Computing others
Y_test = pd.DataFrame(data=Y_test, columns=["q_abs", "q_sca", "g"])

wavelength= df_testing['wavelength']
fractal_dimension= df_testing['fractal_dimension']
fraction_of_coating= df_testing['fraction_of_coating']
primary_particle_size= df_testing['primary_particle_size']
number_of_primary_particles= df_testing['number_of_primary_particles']
vol_equi_radius_inner= df_testing['vol_equi_radius_inner']
vol_equi_radius_outer= df_testing['vol_equi_radius_outer']
equi_mobility_dia= df_testing['equi_mobility_dia']

mie_epsilon= np.zeros_like(wavelength)+2
length_scale_factor= 2*math.pi/wavelength
m_real_bc= np.zeros_like(wavelength)+2
m_im_bc= np.zeros_like(wavelength)+2
m_real_organics= np.zeros_like(wavelength)+2
m_im_organics= np.zeros_like(wavelength)+2

volume_total = (4 * math.pi * (vol_equi_radius_outer ** 3)) / 3
volume_bc = (4 * math.pi * (vol_equi_radius_inner ** 3)) / 3
volume_organics = volume_total - volume_bc

density_bc= np.zeros_like(wavelength)+1.5
density_organics= np.zeros_like(wavelength)+1.1

mass_bc = volume_bc * density_bc * (1 / 1000000000000000000000)
mass_organics = volume_organics * density_organics * (1 / 1000000000000000000000)
mass_total = mass_bc + mass_organics
mr_total_bc = mass_total / mass_bc
mr_nonbc_bc = mass_organics / mass_bc

q_abs= Y_test['q_abs']
q_sca= Y_test['q_sca']
q_ext = q_abs + q_sca
g= Y_test['g']
c_geo = (math.pi) * ((vol_equi_radius_outer) ** 2)
c_ext = (q_ext * c_geo) / (float(1000000))
c_abs = q_abs * c_geo / (1000000)
c_sca = q_sca * c_geo / (1000000)
ssa = q_sca / q_ext
mac_total = (c_abs) / (mass_total * 1000000000000)
mac_bc = c_abs / (mass_bc * (1000000000000))
mac_organics = c_abs / (mass_organics * (1000000000000))


In [60]:
final=np.stack((wavelength, fractal_dimension,fraction_of_coating, primary_particle_size, number_of_primary_particles, vol_equi_radius_inner, vol_equi_radius_outer, equi_mobility_dia, mie_epsilon, length_scale_factor, m_real_bc,m_im_bc, m_real_organics, m_im_organics, volume_total, volume_bc, volume_organics, density_bc, density_organics, mass_total, mass_organics, mass_bc, mr_total_bc, mr_nonbc_bc, q_ext, q_abs, q_sca, g, c_geo, c_ext, c_abs, c_sca, ssa, mac_total, mac_bc, mac_organics),axis=1)

In [62]:
final[0]

array([ 6.60000000e+02,  1.50000000e+00,  0.00000000e+00,  1.50000000e+01,
        1.00000000e+00,  1.50000000e+01,  1.50000000e+01,  2.38296000e+01,
        2.00000000e+00,  9.51997774e-03,  2.00000000e+00,  2.00000000e+00,
        2.00000000e+00,  2.00000000e+00,  1.41371669e+04,  1.41371669e+04,
        0.00000000e+00,  1.50000000e+00,  1.10000000e+00,  2.12057504e-17,
        0.00000000e+00,  2.12057504e-17,  1.00000000e+00,  0.00000000e+00,
       -2.80181179e-03, -2.70463363e-03, -9.71781628e-05, -3.25589720e-03,
        7.06858347e+02, -1.98048405e-06, -1.91179286e-06, -6.86911955e-08,
        3.46840434e-02, -9.01544544e-02, -9.01544544e-02,            -inf])

In [63]:
final_dataset = pd.DataFrame(data=final, columns=['wavelength', 'fractal_dimension', 'fraction_of_coating', 'primary_particle_size',
                      'number_of_primary_particles', 'vol_equi_radius_inner', 'vol_equi_radius_outer', 'equi_mobility_dia',
                      'mie_epsilon', 'length_scale_factor', 'm_real_bc', 'm_im_bc', 'm_real_organics', 'm_im_organics',
                      'volume_total', 'volume_bc', 'volume_organics', 'density_bc', 'density_organics', 'mass_total', 'mass_organics',
                      'mass_bc', 'mr_total_bc', 'mr_nonbc_bc', 'q_ext', 'q_abs', 'q_sca', 'g', 'c_geo', 'c_ext', 'c_abs', 'c_sca', 'ssa',
                      'mac_total', 'mac_bc', 'mac_organics'])

In [69]:
final_dataset.head()

,wavelength,fractal_dimension,fraction_of_coating,primary_particle_size,number_of_primary_particles,vol_equi_radius_inner,vol_equi_radius_outer,equi_mobility_dia,mie_epsilon,length_scale_factor,...,q_sca,g,c_geo,c_ext,c_abs,c_sca,ssa,mac_total,mac_bc,mac_organics
0,660.0,1.5,0.0,15.0,1.0,15.000000,15.000000,23.829600,2.0,0.00952,...,-0.000097,-0.003256,706.858347,-0.000002,-0.000002,-6.869120e-08,0.034684,-0.090154,-0.090154,-inf
1,660.0,1.5,0.0,15.0,2.0,18.898816,18.898816,33.934547,2.0,0.00952,...,-0.000040,0.002108,1122.067684,0.000006,0.000006,-4.435721e-08,-0.007677,0.137275,0.137275,inf
2,660.0,1.5,0.0,15.0,4.0,23.811016,23.811016,48.324498,2.0,0.00952,...,0.000112,0.017844,1781.171422,0.000028,0.000027,1.991604e-07,0.007220,0.322839,0.322839,inf
3,660.0,1.5,0.0,15.0,6.0,27.256809,27.256809,59.425643,2.0,0.00952,...,0.000300,0.033475,2333.994837,0.000053,0.000053,6.997646e-07,0.013125,0.413535,0.413535,inf
4,660.0,1.5,0.0,15.0,8.0,30.000000,30.000000,68.816510,2.0,0.00952,...,0.000537,0.051801,2827.433388,0.000082,0.000081,1.517188e-06,0.018464,0.475411,0.475411,inf


In [70]:
final_dataset.to_csv('../out.csv', index=False) 

In [71]:
x= pd.read_csv('../out.csv')
x.head()

,wavelength,fractal_dimension,fraction_of_coating,primary_particle_size,number_of_primary_particles,vol_equi_radius_inner,vol_equi_radius_outer,equi_mobility_dia,mie_epsilon,length_scale_factor,...,q_sca,g,c_geo,c_ext,c_abs,c_sca,ssa,mac_total,mac_bc,mac_organics
0,660.0,1.5,0.0,15.0,1.0,15.000000,15.000000,23.829600,2.0,0.00952,...,-0.000097,-0.003256,706.858347,-0.000002,-0.000002,-6.869120e-08,0.034684,-0.090154,-0.090154,-inf
1,660.0,1.5,0.0,15.0,2.0,18.898816,18.898816,33.934547,2.0,0.00952,...,-0.000040,0.002108,1122.067684,0.000006,0.000006,-4.435721e-08,-0.007677,0.137275,0.137275,inf
2,660.0,1.5,0.0,15.0,4.0,23.811016,23.811016,48.324498,2.0,0.00952,...,0.000112,0.017844,1781.171422,0.000028,0.000027,1.991604e-07,0.007220,0.322839,0.322839,inf
3,660.0,1.5,0.0,15.0,6.0,27.256809,27.256809,59.425643,2.0,0.00952,...,0.000300,0.033475,2333.994837,0.000053,0.000053,6.997646e-07,0.013125,0.413535,0.413535,inf
4,660.0,1.5,0.0,15.0,8.0,30.000000,30.000000,68.816510,2.0,0.00952,...,0.000537,0.051801,2827.433388,0.000082,0.000081,1.517188e-06,0.018464,0.475411,0.475411,inf
